Курсовая работа


In [1]:
DEFAULT_TRIP_DURATION = 90
DEFAULT_REST_DURATION = 10
PEAK_HOURS = [("07:00", "09:00"), ("17:00", "19:00")]
PEAK_HOURS_ADDITIONAL_TIME = 5
BUS_COUNT = 2
DRIVERS_PER_GROUP_COUNT = 4

In [2]:
from datetime import datetime, timedelta


def is_peak_hour(current_time: datetime) -> bool:
    for start, end in PEAK_HOURS:
        peak_start = datetime.strptime(start, "%H:%M").time()
        peak_end = datetime.strptime(end, "%H:%M").time()
        if peak_start <= current_time.time() < peak_end:
            return True
    return False


def get_trip_duration(start_time: datetime, base_duration: int = DEFAULT_TRIP_DURATION) -> int:
    """
    Рассчитывает длительность поездки:
    - Увеличивает длительность на 5 минут, если поездка пересекает часы пик.
    """
    end_time = start_time + timedelta(minutes=base_duration)

    if is_peak_hour(start_time) or is_peak_hour(end_time):
        base_duration += 5

    return base_duration


class TimeInterval:
    def __init__(self, start: datetime, end: datetime):
        self.start = start
        self.end = end

    def __repr__(self):
        return f"{self.start.strftime('%H:%M')} - {self.end.strftime('%H:%M')}"

    def overlaps(self, other):
        return self.start < other.end and self.end > other.start


class Bus:
    def __init__(self, id: int, capacity: int = 100, speed: int = 60, type: str = "Электробус"):
        self.id = id
        self.capacity = capacity
        self.speed = speed
        self.type = type


class Driver:
    def __init__(self, id: int, bus_id: int, work_time: [(TimeInterval, int)] = None, rest_time: [TimeInterval] = None, type: str = "12_hr", expirience: int = 0):
        self.id = id
        self.bus_id = bus_id
        self.work_time = work_time if  work_time is not None else []
        self.rest_time = rest_time if rest_time is not None else []
        self.type = type
        self.expirience = expirience

    def get_trip_intevals(self):
        return [interval for interval, bus_id in self.work_time]

    def get_last_trip_inteval(self):
        return self.get_trip_intevals()[-1] if self.get_trip_intevals() else None

    def get_work_hours(self):
        if self.type == "12_hr":
            return 12
        elif self.type == "8_hr":
            return 8

        return None

    def get_rest_minutes(self):
        if self.type == "12_hr":
            return 10
        elif self.type == "8_hr":
            return 60
        return None


    def generate_schedule(self, start_time: str):
        self.work_time.clear()
        self.rest_time.clear()

        start = datetime.strptime(start_time, "%H:%M")
        end_time = start + timedelta(hours=self.get_work_hours())

        current_time = start
        trip_count = 0

        while current_time < end_time:
            trip_duration = get_trip_duration(current_time)
            trip_start = current_time
            trip_end = current_time + timedelta(minutes=trip_duration)
            
            if trip_end.time() < trip_start.time():

                self.work_time.append((TimeInterval(trip_start, trip_end), self.bus_id))
                break

            self.work_time.append((TimeInterval(trip_start, trip_end), self.bus_id))

            trip_count += 1
            current_time = trip_end

            if self.type == "12_hr" and trip_count % 2 == 0:
                rest_start = current_time
                rest_end = current_time + timedelta(minutes=self.get_rest_minutes())

                if rest_end.time() < rest_start.time():
                    rest_end = datetime.strptime("00:00", "%H:%M").time()

                    self.rest_time.append(TimeInterval(rest_start, rest_end))
                    break
                
                self.rest_time.append(TimeInterval(rest_start, rest_end))
                current_time = rest_end


In [3]:
def check_continuous_coverage(intervals):
    """
    Проверяет, обеспечивает ли расписание непрерывное покрытие с 00:00 до 00:00 следующего дня.
    """
    intervals.sort(key=lambda x: x.start) 


    for i in range(len(intervals) - 1):
        if intervals[i].end < intervals[i + 1].start and intervals[i].start < intervals[i].end:
            return False, intervals[i].end.strftime("%H:%M"), intervals[i + 1].start.strftime("%H:%M")

    if intervals[-1].end.time() < datetime.strptime("23:59", "%H:%M").time() and intervals[-1].start.time() < intervals[-1].end.time():
        return False, intervals[-1].end.strftime("%H:%M"), "23:59"

    return True, None, None

def get_all_trips_intervals(drivers):
    intervals = []

    for driver in drivers:
        intervals.extend(driver.get_trip_intevals())

    return intervals



def find_valid_schedule(drivers, drivers_start_time):
    # Генерация всех возможных временных значений с шагом 5 минут
    start_time = datetime.strptime("00:00", "%H:%M")
    end_time = datetime.strptime("23:55", "%H:%M")
    time_step = timedelta(minutes=35)
    
    all_times = []
    current_time = start_time
    while current_time <= end_time:
        all_times.append(current_time.strftime("%H:%M"))
        current_time += time_step


    for time_2 in all_times:  # Сначала меняем время для водителя 4
        drivers_start_time[2] = time_2
        for time_3 in all_times:  # Затем для водителя 3
            drivers_start_time[3] = time_3
            for time_4 in all_times:  # В конце для водителя 2
                drivers_start_time[4] = time_4

                
                # Генерация расписания для всех водителей
                for driver in drivers:
                    driver.generate_schedule(drivers_start_time[driver.id])

                # Проверка непрерывного покрытия
                all_intervals = get_all_trips_intervals(drivers)
                is_continuous_coverage, start_time, end_time = check_continuous_coverage(all_intervals)

                # Проверка дополнительных условий
                driver_3_last_trip = drivers[2].get_last_trip_inteval()
                if (
                    is_continuous_coverage
                    and driver_3_last_trip
                    and driver_3_last_trip.end.time() < datetime.strptime("23:45", "%H:%M").time()
                    and driver_3_last_trip.start.time() < driver_3_last_trip.end.time()
                ):
                    # Успешный случай
                    return True

    # Если ни одна комбинация не подошла
    return False



In [4]:
buses = [Bus(id=x) for x in range(1, BUS_COUNT + 1)]
drivers = [Driver(id=driver_id, bus_id=(2 if driver_id % 2 == 0 else 1)) for driver_id in range(1, DRIVERS_PER_GROUP_COUNT + 1)]

# Начальные времена старта
drivers_start_time = {
    1: "00:00",
    2: "00:00",
    3: "00:00",
    4: "00:00"
}

# Поиск валидного расписания
if find_valid_schedule(drivers, drivers_start_time):
# if test(drivers_start_time):

    # Вывод результатов
    for d in drivers:
        for interval in d.get_trip_intevals():
            print(f"Водитель {d.id}, поездка: {interval}, автобус: №{d.bus_id}")
        for interval in d.rest_time:
            print(f"Водитель {d.id}, отдых: {interval}")
else:
    print("Не удалось найти валидное расписание")

Водитель 1, поездка: 00:00 - 01:30, автобус: №1
Водитель 1, поездка: 01:30 - 03:00, автобус: №1
Водитель 1, поездка: 03:10 - 04:40, автобус: №1
Водитель 1, поездка: 04:40 - 06:10, автобус: №1
Водитель 1, поездка: 06:20 - 07:55, автобус: №1
Водитель 1, поездка: 07:55 - 09:30, автобус: №1
Водитель 1, поездка: 09:40 - 11:10, автобус: №1
Водитель 1, поездка: 11:10 - 12:40, автобус: №1
Водитель 1, отдых: 03:00 - 03:10
Водитель 1, отдых: 06:10 - 06:20
Водитель 1, отдых: 09:30 - 09:40
Водитель 1, отдых: 12:40 - 12:50
Водитель 2, поездка: 00:35 - 02:05, автобус: №2
Водитель 2, поездка: 02:05 - 03:35, автобус: №2
Водитель 2, поездка: 03:45 - 05:15, автобус: №2
Водитель 2, поездка: 05:15 - 06:45, автобус: №2
Водитель 2, поездка: 06:55 - 08:30, автобус: №2
Водитель 2, поездка: 08:30 - 10:05, автобус: №2
Водитель 2, поездка: 10:15 - 11:45, автобус: №2
Водитель 2, поездка: 11:45 - 13:15, автобус: №2
Водитель 2, отдых: 03:35 - 03:45
Водитель 2, отдых: 06:45 - 06:55
Водитель 2, отдых: 10:05 - 10:15
В

In [5]:
import pandas as pd


# Создаем 12 водителей (3 группы по 4 водителя)
all_drivers = []
for group in range(3):
    for driver_id in range(1, 5):
        current_id = driver_id + (group * 4)
        bus_id = 2 if driver_id % 2 == 0 else 1
        all_drivers.append(Driver(id=current_id, bus_id=bus_id))

# График работы для каждой группы
work_schedule = {
    1: ['Понедельник', 'Четверг', 'Воскресенье'],
    2: ['Вторник', 'Пятница'],
    3: ['Среда', 'Суббота']
}

# Создаем структуру для таблицы
days = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
columns = pd.MultiIndex.from_product([days, ['График работы', 'Время отдыха']])

# Создаем данные для таблицы
data = []
for driver in all_drivers:
    group_num = (driver.id - 1) // 4 + 1
    
    # Генерируем расписание для водителя
    driver.generate_schedule(drivers_start_time[driver.id % 4 if driver.id % 4 != 0 else 4])
    
    row_data = {}
    
    # Добавляем информацию о водителе
    driver_info = f"Водитель №{driver.id}\nАвтобус №{driver.bus_id}\nГруппа {group_num}"
    row_data['Информация'] = driver_info
    
    for day in days:
        if day in work_schedule[group_num]:
            work_times = [f"{interval.start.strftime('%H:%M')}-{interval.end.strftime('%H:%M')}" 
                         for interval, _ in driver.work_time]
            rest_times = [f"{interval.start.strftime('%H:%M')}-{interval.end.strftime('%H:%M')}" 
                         for interval in driver.rest_time]
            
            row_data[(day, 'График работы')] = '\n'.join(work_times)
            row_data[(day, 'Время отдыха')] = '\n'.join(rest_times)
        else:
            row_data[(day, 'График работы')] = 'Выходной'
            row_data[(day, 'Время отдыха')] = '---'
    
    data.append(row_data)

# Создаем DataFrame
df = pd.DataFrame(data)
df.set_index('Информация', inplace=True)

# Настраиваем отображение
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Применяем стили для лучшей читаемости
styled_df = df.style.set_properties(**{
    'text-align': 'center',
    'white-space': 'pre-wrap'
}).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': '', 'props': [('border', '1px solid black')]},
    {'selector': 'th.col_heading', 'props': [('font-size', '11pt')]},
    {'selector': 'th.col_heading.level0', 'props': [('font-size', '12pt'), ('background-color', '#f0f0f0')]},
])

print("\nРасписание работы водителей на неделю:")
display(styled_df)


Расписание работы водителей на неделю:


,"('Понедельник', 'График работы')","('Понедельник', 'Время отдыха')","('Вторник', 'График работы')","('Вторник', 'Время отдыха')","('Среда', 'График работы')","('Среда', 'Время отдыха')","('Четверг', 'График работы')","('Четверг', 'Время отдыха')","('Пятница', 'График работы')","('Пятница', 'Время отдыха')","('Суббота', 'График работы')","('Суббота', 'Время отдыха')","('Воскресенье', 'График работы')","('Воскресенье', 'Время отдыха')"
Информация,,,,,,,,,,,,,,
Водитель №1 Автобус №1 Группа 1,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50
Водитель №2 Автобус №2 Группа 1,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25
Водитель №3 Автобус №1 Группа 1,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40
Водитель №4 Автобус №2 Группа 1,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20
Водитель №5 Автобус №1 Группа 2,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---,00:00-01:30 01:30-03:00 03:10-04:40 04:40-06:10 06:20-07:55 07:55-09:30 09:40-11:10 11:10-12:40,03:00-03:10 06:10-06:20 09:30-09:40 12:40-12:50,Выходной,---,Выходной,---
Водитель №6 Автобус №2 Группа 2,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---,00:35-02:05 02:05-03:35 03:45-05:15 05:15-06:45 06:55-08:30 08:30-10:05 10:15-11:45 11:45-13:15,03:35-03:45 06:45-06:55 10:05-10:15 13:15-13:25,Выходной,---,Выходной,---
Водитель №7 Автобус №1 Группа 2,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---,08:45-10:20 10:20-11:50 12:00-13:30 13:30-15:00 15:10-16:40 16:40-18:15 18:25-20:00 20:00-21:30,11:50-12:00 15:00-15:10 18:15-18:25 21:30-21:40,Выходной,---,Выходной,---
Водитель №8 Автобус №2 Группа 2,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выходной,---,11:40-13:10 13:10-14:40 14:50-16:20 16:20-17:55 18:05-19:40 19:40-21:10 21:20-22:50 22:50-00:20,14:40-14:50 17:55-18:05 21:10-21:20,Выходной,---,Выхо